In [31]:
import json
import requests
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from google import genai
from google.genai import types
import time
from text_preprocessing import preprocess_text
from text_preprocessing import to_lower, keep_alpha_numeric, remove_special_character

## AFL code is 7523
## WBNA 126714
## Need to add other codes in the game to find the most optimal

game_sportsbet = {'wnba': 29909, 'nba': 6927, 'afl': 4165, 'mlb': 2549}
game_pointsbet = {'wnba': 126714, 'nba': 6927, 'afl': 7523, 'mlb': 112658}
while True:
    comp_code = input("Enter Game")
    if comp_code not in game_sportsbet:
        print("Invalid Game Try Again")
    else:
        break
url = f"https://www.sportsbet.com.au/apigw/sportsbook-sports/Sportsbook/Sports/Competitions/{game_sportsbet[comp_code]}?displayType=default&includeTopMarkets=true&eventFilter=all"

print("Scraping all games in SportsBet")


#29909 is WBNA
#6927 is NBA
#4165 is AFL
headers = {
    "User-Agent": "Mozilla/5.0",
    "Accept": "application/json"
}

response = requests.get(url, headers=headers)
response.raise_for_status()

data = response.json()

event_1_codes = []

market_data = []
for event in data.get("events", []):
    event_1_codes.append(event.get('id'))

for event_code in event_1_codes:
    print("Starting with code", event_code)
    response = requests.get(f"https://www.sportsbet.com.au/apigw/sportsbook-sports/Sportsbook/Sports/Events/{event_code}", headers=headers)
    response.raise_for_status()
    market_data.append(response.json())



Scraping all games in SportsBet
Starting with code 9378470
Starting with code 9378469
Starting with code 9378465
Starting with code 9378464
Starting with code 9381615
Starting with code 9381616
Starting with code 9381613
Starting with code 9381614
Starting with code 9381618
Starting with code 9381612
Starting with code 9381622
Starting with code 9381609
Starting with code 9381611
Starting with code 9381621
Starting with code 9381619
Starting with code 9023720
Starting with code 9023732
Starting with code 9023733
Starting with code 9023734
Starting with code 9023735
Starting with code 9023736


In [32]:
# Separate cell so we don't keep calling API
#
# print(market_data[0])


tempdata = []
storeddata = []
event_labels = []

for event in market_data:
    # print(f"-----{event.get('name')}-----")
    for market in event.get("marketList", []):
        tempdata.append(f"{market.get("name")} ({event.get("name")})")
        event_labels.append(f"{market.get("name")} ({event.get("name")})")
        # print(f"---{market.get('name')}---")
        for selection in market.get("selections", []):
            # print(f"{selection.get("name")}, {selection.get("price", []).get("winPrice")}")
            tempdata.append([selection.get("name"), selection.get("price",[]).get("winPrice")])
        storeddata.append(tempdata)
        tempdata = []

print(storeddata)

[['German Marquez Outs Recorded (Chicago White Sox (J Cannon) At Colorado Rockies (G Marquez))', ['Over 15.5', 1.83], ['Under 15.5', 1.83]], ['Specials Bottom 8th (Chicago White Sox (J Cannon) At Colorado Rockies (G Marquez))', ['3+ Strikeouts', 8.5], ['1+ Home Run', 5.3], ['1+ Stolen Base', 26], ['1+ Extra Base Hit (Double/Triple/Home Run)', 2.9], ['3 Up 3 Down', 2.62], ['1+ Grand Slam', 51]], ['Run Line (Chicago White Sox (J Cannon) At Colorado Rockies (G Marquez))', ['Chicago White Sox', 1.81], ['Colorado Rockies', 1.97]], ['Total Runs (Chicago White Sox (J Cannon) At Colorado Rockies (G Marquez))', ['Over', 2.2], ['Under', 1.65]], ['Money Line / Total Runs Double (Chicago White Sox (J Cannon) At Colorado Rockies (G Marquez))', ['Chicago White Sox / Over 11.5', 1.3], ['Chicago White Sox / Under 11.5', 3.45], ['Colorado Rockies / Over 11.5', 61]], ['Run Line / Total Runs Double (Chicago White Sox (J Cannon) At Colorado Rockies (G Marquez))', ['Chicago White Sox -1.5 / Over 11.5', 1.3

In [33]:
print(storeddata[0])

['German Marquez Outs Recorded (Chicago White Sox (J Cannon) At Colorado Rockies (G Marquez))', ['Over 15.5', 1.83], ['Under 15.5', 1.83]]


In [34]:

import json
import requests

url = f"https://api.au.pointsbet.com/api/mes/v3/events/featured/competition/{game_pointsbet[comp_code]}"

print("Scraping all pointsbet games")

#6927 is NBA
#4165 is AFL
headers = {
    "User-Agent": "Mozilla/5.0",
    "Accept": "application/json"
}

response = requests.get(url, headers=headers)
response.raise_for_status()

event_data_2 = response.json()
event_codes_2 = []
market_data_2 = []


for event_code in event_data_2.get("events", []):
    print(event_code.get("key"))
    event_codes_2.append(event_code.get("key"))

print(event_codes_2)

# In PointsBet the individual markets are spread out across different API calls based on the sepcific game
for event_code in event_codes_2:
    print("Starting with code", event_code)
    response = requests.get(f"https://api.au.pointsbet.com/api/mes/v3/events/{event_code}", headers=headers)
    response.raise_for_status()
    market_data_2.append(response.json())



Scraping all pointsbet games
2339709
2339720
2339723
2339724
2340291
2340339
2340341
2340344
2340343
2340370
2340393
2340514
2340513
2340512
2340574
2340708
['2339709', '2339720', '2339723', '2339724', '2340291', '2340339', '2340341', '2340344', '2340343', '2340370', '2340393', '2340514', '2340513', '2340512', '2340574', '2340708']
Starting with code 2339709
Starting with code 2339720
Starting with code 2339723
Starting with code 2339724
Starting with code 2340291
Starting with code 2340339
Starting with code 2340341
Starting with code 2340344
Starting with code 2340343
Starting with code 2340370
Starting with code 2340393
Starting with code 2340514
Starting with code 2340513
Starting with code 2340512
Starting with code 2340574
Starting with code 2340708


In [35]:

tempdata = []
storeddata_2 = []
event_labels_2 = []


# https://api.au.pointsbet.com/api/mes/v3/events/featured/competition/7523
## AFL code is 7523
## WBNA 126714



for event in market_data_2:
    # print(f"-----{event.get('name')}-----")
    for market_type in ["markets", "fixedOddsMarkets", "specialFixedOddsMarkets"]:
        # print(f"{market_type}")
        for market in event.get(market_type, []):
            tempdata.append(market.get("name"))
            event_labels_2.append(market.get("name"))
            for outcomes in market.get("outcomes", []):
                tempdata.append([outcomes.get("name"), outcomes.get("price")])
            # print(tempdata)
            storeddata_2.append(tempdata)
            tempdata = []

print(storeddata_2)



[['Moneyline (Chicago White Sox @ Colorado Rockies)', ['Colorado Rockies', 21.0], ['Chicago White Sox', 1.005]], ['Run Line (Chicago White Sox @ Colorado Rockies)', ['Colorado Rockies +6.5', 2.0], ['Chicago White Sox -6.5', 1.77]], ['Total Runs (Chicago White Sox @ Colorado Rockies)', ['Over 13.5', 1.9], ['Under 13.5', 1.83]], ['Colorado Rockies Total Runs (Chicago White Sox @ Colorado Rockies)', ['Over 2.5', 1.74], ['Under 2.5', 2.05]], ['Alternate Total Runs (Chicago White Sox @ Colorado Rockies)', ['Over 10.5', 1.1], ['Under 10.5', 6.5], ['Over 11.5', 1.29], ['Under 11.5', 3.4], ['Over 12.5', 1.53], ['Under 12.5', 2.45], ['Over 14.5', 2.5], ['Under 14.5', 1.5], ['Over 15.5', 3.4], ['Under 15.5', 1.29], ['Over 16.5', 4.5], ['Under 16.5', 1.18], ['Over 17.5', 6.5], ['Under 17.5', 1.1]], ['Alternate Total Runs - 3-Way (Chicago White Sox @ Colorado Rockies)', ['Over 12', 1.53], ['Under 12', 3.4], ['Draw 12', 6.0], ['Over 14', 2.5], ['Under 14', 1.83], ['Draw 14', 6.5]], ['Total Runs - 3

In [36]:

for eachevent in storeddata:
    print(eachevent)
    print("\n")


print(len(event_labels_2))
print(len(event_labels))

['German Marquez Outs Recorded (Chicago White Sox (J Cannon) At Colorado Rockies (G Marquez))', ['Over 15.5', 1.83], ['Under 15.5', 1.83]]


['Specials Bottom 8th (Chicago White Sox (J Cannon) At Colorado Rockies (G Marquez))', ['3+ Strikeouts', 8.5], ['1+ Home Run', 5.3], ['1+ Stolen Base', 26], ['1+ Extra Base Hit (Double/Triple/Home Run)', 2.9], ['3 Up 3 Down', 2.62], ['1+ Grand Slam', 51]]


['Run Line (Chicago White Sox (J Cannon) At Colorado Rockies (G Marquez))', ['Chicago White Sox', 1.81], ['Colorado Rockies', 1.97]]


['Total Runs (Chicago White Sox (J Cannon) At Colorado Rockies (G Marquez))', ['Over', 2.2], ['Under', 1.65]]


['Money Line / Total Runs Double (Chicago White Sox (J Cannon) At Colorado Rockies (G Marquez))', ['Chicago White Sox / Over 11.5', 1.3], ['Chicago White Sox / Under 11.5', 3.45], ['Colorado Rockies / Over 11.5', 61]]


['Run Line / Total Runs Double (Chicago White Sox (J Cannon) At Colorado Rockies (G Marquez))', ['Chicago White Sox -1.5 / Over 11.5',

In [37]:

for eachevent in storeddata_2:
    print(eachevent)
    print("\n")

['Moneyline (Chicago White Sox @ Colorado Rockies)', ['Colorado Rockies', 21.0], ['Chicago White Sox', 1.005]]


['Run Line (Chicago White Sox @ Colorado Rockies)', ['Colorado Rockies +6.5', 2.0], ['Chicago White Sox -6.5', 1.77]]


['Total Runs (Chicago White Sox @ Colorado Rockies)', ['Over 13.5', 1.9], ['Under 13.5', 1.83]]


['Colorado Rockies Total Runs (Chicago White Sox @ Colorado Rockies)', ['Over 2.5', 1.74], ['Under 2.5', 2.05]]


['Alternate Total Runs (Chicago White Sox @ Colorado Rockies)', ['Over 10.5', 1.1], ['Under 10.5', 6.5], ['Over 11.5', 1.29], ['Under 11.5', 3.4], ['Over 12.5', 1.53], ['Under 12.5', 2.45], ['Over 14.5', 2.5], ['Under 14.5', 1.5], ['Over 15.5', 3.4], ['Under 15.5', 1.29], ['Over 16.5', 4.5], ['Under 16.5', 1.18], ['Over 17.5', 6.5], ['Under 17.5', 1.1]]


['Alternate Total Runs - 3-Way (Chicago White Sox @ Colorado Rockies)', ['Over 12', 1.53], ['Under 12', 3.4], ['Draw 12', 6.0], ['Over 14', 2.5], ['Under 14', 1.83], ['Draw 14', 6.5]]


['Total Run

In [38]:
test = ['Match Betting (Los Angeles Sparks At New York Liberty)', 'Total Points (Los Angeles Sparks At New York Liberty)', 'Total (New York Liberty v Los Angeles Sparks)', 'Moneyline (New York Liberty v Los Angeles Sparkt54y645yh45ys)']

#concatenate the two label groups for one api call

model = SentenceTransformer('sentence-transformers/stsb-distilroberta-base-v2', trust_remote_code=True)
embeddings_1 = model.encode(event_labels)
embeddings_2 = model.encode(event_labels_2)


# Google GENAI embeddings (rate limited however and need to pay for plan)
# client = genai.Client(api_key="KEY")
# result = client.models.embed_content(
#     model='gemini-embedding-exp-03-07',
#     contents=combined_list,
#     config=types.EmbedContentConfig(task_type='SEMANTIC_SIMILARITY')
# )
# batch_size = 90
# for i in range(0, len(combined_list)):
#     batch = combined_list[i:i + batch_size]
#     result = client.models.embed_content(
#     model='gemini-embedding-exp-03-07',
#     contents=batch,
#     config=types.EmbedContentConfig(task_type='SEMANTIC_SIMILARITY')
#     )
#     combined_embeddings.extend(batch)
#     time.sleep(100)





In [39]:
matched_labels = []
matched_indexes = []

embed_1_bigger = False

if len(embeddings_1) > len(embeddings_2):
    cos_similarity = cosine_similarity(embeddings_2, embeddings_1)
    for i, row in enumerate(cos_similarity):
        print(event_labels_2[i], 'AND', event_labels[np.argmax(row)])
        matched_labels.append([event_labels_2[i], event_labels[np.argmax(row)]])
        matched_indexes.append([i, int(np.argmax(row))])
        print('-----------')
    embed_1_bigger = True
else:
    cos_similarity = cosine_similarity(embeddings_1, embeddings_2)
    for i, row in enumerate(cos_similarity):
        print(event_labels[i], 'AND', event_labels_2[np.argmax(row)])
        matched_labels.append([event_labels[i], event_labels_2[np.argmax(row)]])
        matched_indexes.append([i, int(np.argmax(row))])
        print('-----------')
print(cos_similarity)


Moneyline (Chicago White Sox @ Colorado Rockies) AND Money Line (Chicago White Sox (S Smith) At Colorado Rockies (C Dollander))
-----------
Run Line (Chicago White Sox @ Colorado Rockies) AND Run Line (Chicago White Sox (J Cannon) At Colorado Rockies (G Marquez))
-----------
Total Runs (Chicago White Sox @ Colorado Rockies) AND Colorado Rockies Total Runs (Chicago White Sox (S Smith) At Colorado Rockies (C Dollander))
-----------
Colorado Rockies Total Runs (Chicago White Sox @ Colorado Rockies) AND Colorado Rockies Total Runs (Chicago White Sox (S Smith) At Colorado Rockies (C Dollander))
-----------
Alternate Total Runs (Chicago White Sox @ Colorado Rockies) AND Alternate Total Runs (Chicago White Sox (S Smith) At Colorado Rockies (C Dollander))
-----------
Alternate Total Runs - 3-Way (Chicago White Sox @ Colorado Rockies) AND 3 Innings Total Runs (Chicago White Sox (S Smith) At Colorado Rockies (C Dollander))
-----------
Total Runs - 3-Way (Chicago White Sox @ Colorado Rockies) AND

In [40]:
# print(matched_indexes)

possible_arbs = []
if embed_1_bigger:
    for index in matched_indexes:
        print([storeddata_2[index[0]], '<---------->', storeddata[index[1]]])
        possible_arbs.append([storeddata_2[index[0]], storeddata[index[1]]])
        print('\n')
else:
    for index in matched_indexes:
        print([storeddata[index[0]], '<---------->', storeddata_2[index[1]]])
        possible_arbs.append([storeddata[index[0]], storeddata_2[index[1]]])
        print('\n')


# For now only deal with single bet lines



[['Moneyline (Chicago White Sox @ Colorado Rockies)', ['Colorado Rockies', 21.0], ['Chicago White Sox', 1.005]], '<---------->', ['Money Line (Chicago White Sox (S Smith) At Colorado Rockies (C Dollander))', ['Chicago White Sox', 1.82], ['Colorado Rockies', 1.98]]]


[['Run Line (Chicago White Sox @ Colorado Rockies)', ['Colorado Rockies +6.5', 2.0], ['Chicago White Sox -6.5', 1.77]], '<---------->', ['Run Line (Chicago White Sox (J Cannon) At Colorado Rockies (G Marquez))', ['Chicago White Sox', 1.81], ['Colorado Rockies', 1.97]]]


[['Total Runs (Chicago White Sox @ Colorado Rockies)', ['Over 13.5', 1.9], ['Under 13.5', 1.83]], '<---------->', ['Colorado Rockies Total Runs (Chicago White Sox (S Smith) At Colorado Rockies (C Dollander))', ['Over', 1.72], ['Under', 2.13]]]


[['Colorado Rockies Total Runs (Chicago White Sox @ Colorado Rockies)', ['Over 2.5', 1.74], ['Under 2.5', 2.05]], '<---------->', ['Colorado Rockies Total Runs (Chicago White Sox (S Smith) At Colorado Rockies (C Do

In [41]:
# Preprocessing to allow simple matching now


# For now just search for arbitrages between bet lines with only 2 outcomes
# Later need to add the multi odds

possible_arbs_filtered = []
temp_data = []

for event in possible_arbs:
    if all(len(pair) <= 3 for pair in event):
        possible_arbs_filtered.append(event)

for event in possible_arbs:
    for pair in event:
        print(pair)

#
# for event in possible_arbs:
#     for pair in event:
#         print(f"{pair}")
#         print(len(pair))
#         print('\n')
#     print("HIHIHI")

print(possible_arbs_filtered)

['Moneyline (Chicago White Sox @ Colorado Rockies)', ['Colorado Rockies', 21.0], ['Chicago White Sox', 1.005]]
['Money Line (Chicago White Sox (S Smith) At Colorado Rockies (C Dollander))', ['Chicago White Sox', 1.82], ['Colorado Rockies', 1.98]]
['Run Line (Chicago White Sox @ Colorado Rockies)', ['Colorado Rockies +6.5', 2.0], ['Chicago White Sox -6.5', 1.77]]
['Run Line (Chicago White Sox (J Cannon) At Colorado Rockies (G Marquez))', ['Chicago White Sox', 1.81], ['Colorado Rockies', 1.97]]
['Total Runs (Chicago White Sox @ Colorado Rockies)', ['Over 13.5', 1.9], ['Under 13.5', 1.83]]
['Colorado Rockies Total Runs (Chicago White Sox (S Smith) At Colorado Rockies (C Dollander))', ['Over', 1.72], ['Under', 2.13]]
['Colorado Rockies Total Runs (Chicago White Sox @ Colorado Rockies)', ['Over 2.5', 1.74], ['Under 2.5', 2.05]]
['Colorado Rockies Total Runs (Chicago White Sox (S Smith) At Colorado Rockies (C Dollander))', ['Over', 1.72], ['Under', 2.13]]
['Alternate Total Runs (Chicago Whit

In [42]:


# Possible abrbitrage finder
'''
This model assumes that the order of the bet lines are the same, that is SportsBet and PointsBet both show Over and Unders in the same list order
the JSON file from brief inspection also supported this theory
'''

for arb in possible_arbs_filtered:
    try:
        if arb[0][2] is None or arb[1][2] is None:
            continue
    except IndexError:
        continue
    # print(f"first site {arb[0][1]} AND {arb[0][2]} second site {arb[1][1]}AND {arb[1][2]}")
    arb_rate = (1/arb[0][1][1]) + (1/arb[1][2][1])
    arb_rate_2 = (1/arb[0][2][1]) + (1/arb[1][1][1])
    if arb_rate < 1:
        print(f"ARB FOUND {arb} AT RATE {arb_rate}")
        print(f"calculated by 1/{arb[0][1][1]} + 1/{arb[1][2][1]}")
        print('\n')
    elif arb_rate_2 < 1:
        print(f"ARB FOUND {arb} AT RATE {arb_rate_2}")
        print(f"calculated by 1/{arb[0][2][1]} + 1/{arb[1][1][1]}")
        print('\n')

ARB FOUND [['Moneyline (Chicago White Sox @ Colorado Rockies)', ['Colorado Rockies', 21.0], ['Chicago White Sox', 1.005]], ['Money Line (Chicago White Sox (S Smith) At Colorado Rockies (C Dollander))', ['Chicago White Sox', 1.82], ['Colorado Rockies', 1.98]]] AT RATE 0.5526695526695526
calculated by 1/21.0 + 1/1.98


ARB FOUND [['Total Runs (Chicago White Sox @ Colorado Rockies)', ['Over 13.5', 1.9], ['Under 13.5', 1.83]], ['Colorado Rockies Total Runs (Chicago White Sox (S Smith) At Colorado Rockies (C Dollander))', ['Over', 1.72], ['Under', 2.13]]] AT RATE 0.9957993575488016
calculated by 1/1.9 + 1/2.13


ARB FOUND [['8th Inning - Total Runs Odd/Even (Chicago White Sox @ Colorado Rockies)', ['Odd', 2.1], ['Even', 1.71]], ['8th Inning Runs Odd/Even (Chicago White Sox (S Smith) At Colorado Rockies (C Dollander))', ['Odd', 2.76], ['Even', 1.45]]] AT RATE 0.9471141622171371
calculated by 1/1.71 + 1/2.76


ARB FOUND [['K. Teel To Reach Base In 8th Inning (Chicago White Sox @ Colorado Rock